In [1]:
# https://www.tensorflow.org/
# https://docs.opencv.org/4.x/
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython import display

In [2]:
folder_path = './datasets/dataset_cleaned'
out_path = './datasets/out'
assert os.path.exists(folder_path)

In [3]:

def detect_edges(image):
    image = cv.Canny(image, 120, 200, apertureSize=3)
    return image

def detect_edges_clahe(image):
    clahe = cv.createCLAHE(2, (4,4))
    (ib, ig, ir) = cv.split(image)
    image[:, :, 0], image[:, :, 1], image[:, :, 2] = clahe.apply(ib), clahe.apply(ig), clahe.apply(ir)
    image = cv.Canny(image, 120, 200, apertureSize=3)
    return image
    
def detect_edges_clahe_gray(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(2, (4,4))
    image = clahe.apply(image)
    image = cv.Canny(image, 120, 200, apertureSize=3)
    return image

def detect_contours_adaptive_thresh(image):
    image = cv.GaussianBlur(image, (3, 3), 0)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,-5)
    contours, hierarchy = cv.findContours(gray, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    image = cv.drawContours(np.zeros((len(image),len(image[0]),len(image[0,0]))), contours[:], -1, (255,255,255), 1)
    return image

def detect_contours_clahe(image):
    image = cv.GaussianBlur(image, (3, 3), 0)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(2, (20,20))
    gray = clahe.apply(gray)
    _, gray = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
    contours, hierarchy = cv.findContours(gray, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)
    image = cv.drawContours(np.zeros((len(image),len(image[0]),len(image[0,0]))), contours[:], -1, (255,255,255), 1)
    return image

def filter_contours(contours):
    filter_array = []
    contours = np.array(contours)
    for c in contours:
        if cv.contourArea(c) < 250:
            filter_array.append(False)
            continue
        # Check by relative area, contour points total, contour points per sqrt(area), shape
        filter_array.append(True)
    return contours[filter_array]

In [4]:
def process_image(image):
    image = detect_contours_clahe(image)
    return image

def display_image(image, single: bool) -> None:
    plt.imshow(image, cmap='gray')
    # plt.imshow(image)
    if single:
        display.clear_output()
        display.display(plt.gcf())
    else:
        plt.show()
        
for image_path in os.scandir(folder_path):
    image = cv.imread(os.path.join(folder_path,image_path.name))
    
    image = process_image(image)
    
    # display_image(image, True)
    cv.imwrite(os.path.join(out_path,image_path.name), image)
    
    # if input() == 'q':
        # break

C:\Users\Maciej\AppData\Local\Temp\ipykernel_8344\2668395315.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  contours = np.array(contours)


KeyboardInterrupt: 